In [ ]:
!pip install transformers
!pip install datasets
!pip list
!pip install rouge_score
!pip install bert_score

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Package                  Version            
------------------------ -------------------
accelerate               0.27.2             
aiohttp                  3.9.3              
aiosignal                1.3.1              
alembic                  1.4.3              
appdirs                  1.4.4              
argon2-cffi              20.1.0             
async-generator          1.10               
async-timeout            4.0.3              
attrs                    20.3.0             
backcall                 0.2.0              
batchspawner             1.0.1              
bleach                   3.2.1              
blinker                  1.4                
certifi                  2020.12.5          
certipy                  0.1.3              
cffi                     1.14.0             
chardet                  3.0.4    

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
from datasets import load_dataset

In [ ]:
# By Madhan Mohan

# Task 1

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

prompts = [
    "With a heavy heart,",
    "Standing on the edge,",
    "Surrounded by strangers,",
    "Caught in a dilemma,",
    "At the crossroads,"
]






In [ ]:
num_beams = 5


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i, prompt in enumerate(prompts):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    beam_search_config = model.generate(
        input_ids, 
        max_length=30, 
        num_beams=num_beams,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    
    output = tokenizer.batch_decode(beam_search_config, skip_special_tokens=True)
    
    encodings = tokenizer(output, return_tensors="pt").to(device)
    print("Output:", output)
    print()
    
    # Attribution of code: https://huggingface.co/docs/transformers/main/en/perplexity
    max_length = model.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        encodings = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = encodings.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(encodings, labels=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    print("Generated Text:", output)
    print("Perplexity: ", ppl)



In [ ]:
# Task 2

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

dataset = load_dataset('cnn_dailymail', '1.0.0')

test_dataset = dataset["test"]

sentence_list = []
reference_list = []

sentence_list = [item["article"] for item in test_dataset[:50]]
sentence_list = [item["highlights"] for item in test_dataset[:50]]

prediction_list = []
for i in range(len(sentence_list)):
    input_ids = tokenizer(sentence_list[i], return_tensors="pt", max_length=1024, truncation=True).input_ids
    res = model.generate(input_ids, num_beams=5, do_sample=False, max_length=30[0])
    prediction_list.append(tokenizer.batch_decode(res, skip_special_tokens=True)[0])




In [ ]:
print("Input Text")
for i in range(len(sentence_list)):
    print(sentence_list[i][:1024])
    print()

In [ ]:
print("Reference Text")
for i in range(len(sentence_list)):
    print(reference_list[i])
    print()

In [ ]:
# Task 3

import evaluate

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=prediction_list, references=reference_list, lang="en")
print("Rouge Results: ", results)




In [ ]:
bert = evaluate.load("bertscore")
results = rouge.compute(predictions=prediction_list, references=reference_list, lang="en")
print("Bert Results: ", results)